<a href="https://colab.research.google.com/github/dkurbatovv/Python/blob/main/Titanic_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
!mkdir -p ~/.kaggle/
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d hesh97/titanicdataset-traincsv

In [ ]:
! unzip titanicdataset-traincsv.zip

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import glob as gb


In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df = df.drop('PassengerId', axis = 1)

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
plt.figure(figsize = (12,10))
plt.bar(df.columns, df.isna().sum())

In [ ]:
sns.countplot(data = df, x = 'Survived')

In [ ]:
sns.countplot(data = df, x = 'Embarked')

In [ ]:
sns.countplot(data = df, x = 'Embarked', hue = 'Sex')

In [ ]:
sns.countplot(data = df, x = 'Survived', hue = 'Sex')

In [ ]:
sns.countplot("Survived", hue = 'Pclass', data = df)
plt.show()

In [ ]:
sns.countplot('Survived', hue = 'Embarked', data = df)
plt.show()

In [ ]:
sns.boxplot(x = 'Age', data = df)
plt.show()

In [ ]:
df.columns

In [ ]:
sns.boxplot(x = 'Fare', data = df)
plt.show()

In [ ]:
plt.hist(data = df, x = 'Age', bins = 8)

In [ ]:
plt.figure(figsize = (10,4))
plt.hist(df.Fare, bins = 10, color = 'lime')
plt.xlabel("Fare")
plt.ylabel("Number")
plt.show()

In [ ]:
sns.heatmap(df.corr(), annot = True, fmt = '.1f')

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.drop('Cabin', axis = 1)

In [ ]:
df.Age.fillna(df.Age.mean(), inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.Embarked.fillna('S', inplace = True)

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
le_features = ['Sex', 'Embarked']
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['Sex'] = le.fit_transform(df['Sex'])
df['Embarked'] = le.fit_transform(df['Embarked'])

In [ ]:
df.head()


In [ ]:
df['Alone'] = df.SibSp + df.Parch

In [ ]:
df['Alone'] = df['Alone'].apply(lambda x: 1 if x == 0 else 0)#ставим 1 если один, и 0 если были братья или родственники 

In [ ]:
df = df.drop(['SibSp', 'Parch'], axis = 1)

In [ ]:
df.head()

In [ ]:
df['Title'] = df.Name.str.extract('([A-Za-z]+)\.', expand = False)
df.drop('Name', axis = 1, inplace = True)

In [ ]:
df.head()

In [ ]:
df.Title.value_counts()

In [ ]:
df.Title.unique()

In [ ]:
title_features = ['Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'Countess',
       'Jonkheer']

In [ ]:
df.Title = df.Title.replace(title_features, 'Rare')

In [ ]:
df.Title.unique()

In [ ]:
df.Title = le.fit_transform(df['Title'])

In [ ]:
df.head()

In [ ]:
df = df.drop('Ticket', axis = 1)

In [ ]:
df.head()

In [ ]:
print(df.Age.value_counts())

In [ ]:
df.loc[ df['Age'] <= 16, 'Age'] = 0
df.loc[(df['Age'] > 16) & (df['Age'] <= 32), 'Age'] = 1
df.loc[(df['Age'] > 32) & (df['Age'] <= 48), 'Age'] = 2
df.loc[(df['Age'] > 48) & (df['Age'] <= 64), 'Age'] = 3
df.loc[ df['Age'] > 64, 'Age'] = 4

In [ ]:
df.head()

In [ ]:
df.loc[df['Fare'] < 30, 'Fare'] = 1
df.loc[(df['Fare'] >= 30) & (df['Fare'] < 50),'Fare'] = 2
df.loc[(df['Fare'] >= 50) & (df['Fare'] < 100),'Fare'] = 3
df.loc[(df['Fare'] >= 100),'Fare'] = 4

In [ ]:
df.Fare = df.Fare.astype(int)
df.Age = df.Age.astype(int)

In [ ]:
df.info()


In [ ]:
sns.heatmap(df.corr(), annot = True, fmt = '.1f')

In [ ]:
df.head()


In [ ]:
X = df.drop('Survived', axis = 1)
y = df['Survived']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
import tensorflow

In [ ]:
model = Sequential()

model.add(Dense(units = 32, input_shape = (7,), activation = 'relu'))
model.add(Dense(units = 64, activation = 'relu', kernel_initializer = 'he_normal', use_bias = False))
model.add(tensorflow.keras.layers.BatchNormalization())

model.add(Dense(units = 128, activation = 'relu',kernel_initializer = 'he_normal', use_bias = False))
model.add(Dropout(0.1))

model.add(Dense(units = 64, activation = 'relu',kernel_initializer = 'he_normal', use_bias = False))
model.add(Dropout(0.1))

model.add(Dense(units = 32, activation = 'relu'))
model.add(Dropout(0.15))

model.add(Dense(units = 16, activation = 'relu'))
model.add(Dense(units = 8, activation = 'relu',kernel_initializer = 'he_normal', use_bias = False))
model.add(Dense(units =1 , activation = 'sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs = 100, verbose = 2)

In [ ]:
from sklearn import metrics
Y_pred_rand = (model.predict(X_train) > 0.5).astype(int)
print('Precision : ', np.round(metrics.precision_score(y_train, Y_pred_rand)*100,2))
print('Accuracy : ', np.round(metrics.accuracy_score(y_train, Y_pred_rand)*100,2))
print('Recall : ', np.round(metrics.recall_score(y_train, Y_pred_rand)*100,2))
print('F1 score : ', np.round(metrics.f1_score(y_train, Y_pred_rand)*100,2))
print('AUC : ', np.round(metrics.roc_auc_score(y_train, Y_pred_rand)*100,2))

In [ ]:
matrix = metrics.confusion_matrix(y_train, Y_pred_rand)
sns.heatmap(matrix, annot = True, fmt = 'g')
plt.show()